In [ ]:
import numpy as np
import time
from robot.vrep_robot import VrepRobot
try:
    from robot.evolved_robot import EvolvedRobot
except ImportError as error:
    print(error.__class__.__name__ + ": " + 'DBus works only on linux!')
import vrep.vrep as vrep
from settings import Settings
from vision.tracker import Tracker, get_marker_object
from utility.path_tracking import transform_pos_angle
import math

In [ ]:
vision_thread = Tracker(mid=5,
                        transform=None,
                        mid_aux=0,
                        video_source=-1,
                        capture=False,
                        show=True,
                        debug=False,
                       )

In [ ]:
vision_thread.start()

In [ ]:
thymio = {
    'name': 'thymio',
    'body': 'Thymio',
    'left_motor': 'leftMotor',
    'right_motor': 'rightMotor',
    'sensor': 'Proximity_sensor',
    'num_sensors': 7
}

settings = Settings(thymio)

In [ ]:
vrep.simxFinish(-1)
clientID = vrep.simxStart(
    '127.0.0.1',
    19997,
    True,
    True,
    5000,
    5)

if clientID == -1:
    print('Failed connecting to remote API server')
    print('Program ended')

if (vrep.simxStartSimulation(clientID, vrep.simx_opmode_blocking) == -1):
    print('Failed to start the simulation\n')
    print('Program ended\n')


OP_MODE = vrep.simx_opmode_oneshot_wait
# robot = EvolvedRobot('thymio-II',clientID, None, OP_MODE, None, settings.robot_type)
robot = VrepRobot(clientID, None, OP_MODE, settings.robot_type)

In [ ]:
obstacle = get_marker_object(9)
print(obstacle.realxy())
robot_m = get_marker_object(7)
print(robot_m.realxy())
print(robot_m.orientation())
robot_current_position = robot_m.realxy()[:2]
angle = robot_m.orientation()
print(angle)
if angle >= 0.0 and angle <= math.pi/2:
    theta = (math.pi/2) - angle
else:
    theta = 2*np.pi - angle
    theta += np.pi/2
print(theta)
# update position and orientation of the robot in vrep
position, orientation = transform_pos_angle(
    robot_current_position, theta)
robot.v_set_pos_angle(position, [ 0, 0, theta ])

In [ ]:
robot.v_read_prox()

In [ ]:
robot.v_prox_sensors